# Reconstructing historical portfolio information from Coinbase Pro

Recently I've started doing some very small scale cryptocurrency trading using [Coinbase Pro](https://pro.coinbase.com/). Currently my method of keeping track of my historical orders is by duplicating the information found on the Orders tab. One thing that Coinbase Pro doesn't do however is keep track of your historical portfolio *value* (like Coinbase non-Pro does) or the historical constituents of the portfolio. In this notebook I'll reconstruct these values from a list of trades and deposits made in my main portfolio. Note that currently I only have 25 actions to analyse, so some analysis won't be worth carrying out. I plan on revisiting this in 6 months time or so, to analyse on a larger timescale. 

First, the imports:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

Although we could use the [Coinbase Pro API](https://docs.pro.coinbase.com/) to get our order history, I already have the orders saved as a CSV file, which makes loading it a lot easier.

In [2]:
history = pd.read_csv('data/filledorders.csv', engine='python')
history.head()

,Date,Type,Base,Quote,Amount,Price,Price (ï¿½),Fees,Total Value,Q/ï¿½,GBP Value
0,13/07/20,DEP,LINK,GBP,100.3234,5.8300,5.8300,0.00000,584.89,1.00,ï¿½584.89
1,13/07/20,DEP,XTZ,GBP,27.0834,2.2400,2.2400,0.00000,60.67,1.00,ï¿½60.67
2,13/07/20,DEP,XRP,GBP,124.7508,0.1597,0.1597,0.00000,19.92,1.00,ï¿½19.92
3,13/07/20,DEP,BTC,GBP,0.0255,7386.7500,7386.7500,0.00000,188.70,1.00,ï¿½188.70
4,13/07/20,BUY,ETH,BTC,0.3809,0.0261,192.9419,0.00005,0.01,7386.75,ï¿½73.87


In [3]:
history.shape

(24, 11)

First things first we need to clean up the data. The *Date* column should be converted to datetime and the GBP Value column should be a number.

In [4]:
history['Date'] = pd.to_datetime(history['Date'], format="%d/%m/%y")
history['GBP Value'] = history['GBP Value'].map(lambda x: float(x.strip('ï¿½')))
history.head()

,Date,Type,Base,Quote,Amount,Price,Price (ï¿½),Fees,Total Value,Q/ï¿½,GBP Value
0,2020-07-13,DEP,LINK,GBP,100.3234,5.8300,5.8300,0.00000,584.89,1.00,584.89
1,2020-07-13,DEP,XTZ,GBP,27.0834,2.2400,2.2400,0.00000,60.67,1.00,60.67
2,2020-07-13,DEP,XRP,GBP,124.7508,0.1597,0.1597,0.00000,19.92,1.00,19.92
3,2020-07-13,DEP,BTC,GBP,0.0255,7386.7500,7386.7500,0.00000,188.70,1.00,188.70
4,2020-07-13,BUY,ETH,BTC,0.3809,0.0261,192.9419,0.00005,0.01,7386.75,73.87


## Finding Historical Portfolio Constituents

Firstly, we are going to work out what coins were in my portfolio after each transaction. Hopefully I'll end up with a DataFrame where each column is representing a different coin. This will make it easy to plot. To do this we need to first create the empty dataframe.

In [5]:
cur = (history['Base'].append(history['Quote'])).unique()
print(cur)
print(type(cur))

['LINK' 'XTZ' 'XRP' 'BTC' 'ETH' 'ALGO' 'LTC' 'COMP' 'ZRX' 'GBP' 'EUR']
<class 'numpy.ndarray'>


In [6]:
constituents = pd.DataFrame(columns=(['Date'] + sorted(list(cur))))
constituents

,Date,ALGO,BTC,COMP,ETH,EUR,GBP,LINK,LTC,XRP,XTZ,ZRX


We can safely assume that at `earliest date` - 1, we had no coins whatsoever.

In [9]:
print(history['Date'].min() + pd.DateOffset(days=-1))

2020-07-12 00:00:00
